
# Preprocess Microsoft Academics Graph (MAG) Dataset

Jupyter Notebook for the preprocessing of the Microsoft Academics Graph (MAG) dump.

For this process, the following CSV files are needed: ```ConferenceInstances.txt```, ```ConferenceSeries.txt```, ```Papers.txt```. 
The above files can be found here: https://archive.org/download/mag-2021-06-07/mag/

In particular, the following operations are going to be executed:
* Opening of ConferenceInstances and ConferenceSeries CSVs
* Drop of the useless columns 
* Chuncked Processing of the Papers CSV
    * Drop of the useless columns
    * Drop of papers without DOI
    * Drop of papers from journals and books rows
* Merge with the processed conferences data

Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [1]:
# Libraries Import
import pandas as pd
import platform
import multiprocessing as mp 
import concurrent       
from preprocess_multithread_utils import * 

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [2]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

### Use a Previuously Preprocessed Papers CSV
This can be really useful to save some time using a previously elaborated CSV file. We don't need to repeat the same operations!

**Note**: the CSV needs to be in the same format of the one generated with this script

In [3]:
# Use a Previuously Preprocessed Papers CSV
#
# This can be really useful to save some time using a previously elaborated
# CSV file. We don't need to repeat the same operations!
#
# Note: the CSV needs to be in the same format of the one generated with this script
read_preprocessed_papers = True
preprocessed_papers_csv_path = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_mag_papers.csv'

### Multithreading Settings
Settings needed for the multithreaded queries to gather the missing conferences locations from the DBLP website.

Please specify the number of CPU threads below:

In [4]:
n_cpu_threads = 8 # Number of CPU threads

Special setting for the specific operating systems.

**Note**: Due to the latest MacOS releases' security measures, we need to use the spawn method instead of fork.

In [5]:
print(f"Notebook running on {platform.system()} OS: ")

if platform.system() == "Darwin" or platform.system() == "Windows": # MacOS and windows
    mp_ctx = mp.get_context("spawn")
    print("Spawn method has been set")
    
else: # other unix systems
    mp_ctx = mp.get_context("fork")
    print("Spawn method has been set")

Notebook running on Darwin OS: 
Spawn method has been set


## Preprocess of Conference Instances CSV

In [ ]:
# ******************* CONFERENCE INSTANCES ********************

# Read of the Conference Instances File

# The column names follow the MAG' scheme official documentation
df_mag_conf_instances_col_names = ['ConferenceInstanceID', 'NormalizedName', 'DisplayName', 'ConferenceSeriesID', 'Location', 'OfficialUrl', 'StartDate', 'EndDate', 'AbstractRegistrationDate', 'SubmissionDeadlineDate', 'NotificationDueDate', 'FinalVersionDueDate', 'PageCount', 'PaperFamilyCount', 'CitationCount', 'Latitude', 'Longitude', 'CreatedDate']

df_mag_conf_instances = pd.read_csv(path_file_import + 'ConferenceInstances.txt', sep='\t', names=df_mag_conf_instances_col_names)
df_mag_conf_instances

Here the useless columns are going to be removed from the dataframe.

In [ ]:
# Drop of Conference Instances' Useless Columns
df_mag_conf_instances = df_mag_conf_instances.drop(columns=['OfficialUrl', 'AbstractRegistrationDate', 'SubmissionDeadlineDate', 'NotificationDueDate', 'FinalVersionDueDate', 'PageCount', 'PaperFamilyCount', 'CitationCount', 'Latitude', 'Longitude', 'CreatedDate', 'StartDate', 'EndDate'])
df_mag_conf_instances

Column rename to remove ambiguity for the future joins

In [ ]:
# Column rename to remove ambiguity for the future joins
df_mag_conf_instances.rename(columns={'NormalizedName': 'ConferenceNormalizedName', 'DisplayName': 'ConferenceDisplayName', 'Location': 'ConferenceLocation'}, inplace=True)
df_mag_conf_instances

## Preprocess of Conference Series CSV

In [ ]:
# ******************* CONFERENCE SERIES ********************

# Read of the Conference Series File

# The column names follow the MAG' scheme official documentation
df_mag_conf_series_col_names = ['ConferenceSeriesID', 'Rank', 'NormalizedName', 'DisplayName', 'PaperCount', 'PaperFamilyCount', 'CitationCount', 'CreatedDate']

df_mag_conf_series = pd.read_csv(path_file_import + 'ConferenceSeries.txt', sep='\t', names=df_mag_conf_series_col_names)
df_mag_conf_series

Here the useless columns are going to be removed from the dataframe.

In [ ]:
# Drop of Conference Series' Useless Columns
df_mag_conf_series = df_mag_conf_series.drop(columns=['Rank', 'PaperCount', 'PaperFamilyCount', 'CitationCount', 'CreatedDate'])
df_mag_conf_series

Column rename to remove ambiguity for the future joins

In [ ]:
# Column rename to remove ambiguity for the future joins
df_mag_conf_series.rename(columns={'NormalizedName': 'ConferenceSeriesNormalizedName', 'DisplayName': 'ConferenceSeriesDisplayName'}, inplace=True)
df_mag_conf_series

## Preprocess of Papers CSV
The Papers CSV is going to be processed in chunks, due to its size.

The following operations are going to be executed:
* Drop of the useless columns
* Filtering of papers without DOI
* Filtering papers that are not related to conferences
* Drop of the doctype column
* Write of the processed file on disk (in CSV format)

In [ ]:
# ******************* PAPERS ********************

# Read of previously prerocessed CSV
df_mag_papers = None
if read_preprocessed_papers:
    df_mag_papers = pd.read_csv(preprocessed_papers_csv_path, low_memory=False, index_col=0)
else:
    # The Papers CSV is going to be processed in chunks, due to its size

    # The column names follow the MAG' scheme official documentation
    df_mag_papers_col_names = ['PaperID', 'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle', 'BookTitle', 'Year', 'Date', 'OnlineDate', 'Publisher', 'JournalID', 'ConferenceSeriesID', 'ConferenceInstanceID', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount', 'EstimatedCitation', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'Retracion', 'CreatedDate']

    # List of processed chunks.
    df_mag_papers_list_of_chunks = list()

    # Define of the chunk size
    chunksize = 10 ** 7

    count = 1
    with pd.read_csv(path_file_import + 'Papers.txt', sep='\t', chunksize=chunksize, low_memory=False, on_bad_lines='skip', names=df_mag_papers_col_names) as reader:
        for chunk in reader:

            # Drop of the useless columns
            chunk = chunk.drop(columns=['Rank', 'OnlineDate', 'Publisher', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'Retracion', 'CreatedDate', 'JournalID', 'BookTitle', 'Date'])

            # Filtering of papers without DOI
            chunk = chunk.dropna(subset = ['Doi'])

            # Filtering papers that are not related to conferences
            chunk = chunk[chunk.DocType == 'Conference']

            # Drop of the doctype column
            chunk = chunk.drop(columns=['DocType'])

            # Insert of the resulting chunk in the list 
            df_mag_papers_list_of_chunks.append(chunk)

            print(f'Successfully processed chunk {count} out of around {260000000 / chunksize}')
            count += 1
            break

    # Concatenation of the processed chunks
    df_mag_papers = pd.concat(df_mag_papers_list_of_chunks)

    # Empty the list to free some memory
    df_mag_papers_list_of_chunks = list()

    # Write of the resulting CSV on Disk
    df_mag_papers.to_csv(path_file_export + 'out_mag_papers.csv')
    print(f'Successfully Exported the Preprocessed Papers CSV to {path_file_export}out_mag_papers.csv')

df_mag_papers

## Merge of Conferences and Papers Data

In [ ]:
# ******************* MERGE OF CONFERENCES AND PAPERS DATA ********************

# Merge of conferences and papers data over the conferenceseries id columnn to get the conference series name
# The papers' row that will not match will be preserved
df_mag_preprocessed = pd.merge(df_mag_papers, df_mag_conf_series, on=['ConferenceSeriesID'], how='left')

# Merge of conferences and papers data over the conferenceinstances id columnn
# The papers' row that will not match will be preserved
df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_mag_conf_instances, on=['ConferenceInstanceID'], how='left')

# Drop of the duplicated columns
df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceSeriesID_y'])
df_mag_preprocessed.rename(columns = {'ConferenceSeriesID_x':'ConferenceSeriesID'}, inplace=True)

# Removing broken data (four records seems to have mismatched types in some columns)
df_mag_preprocessed = df_mag_preprocessed.dropna(subset = ['CitationCount'])

df_mag_preprocessed

## Fix of the Mismatched Data Type

Fix of the year data type, that has been interpreted as a float.

In [ ]:
type(df_mag_preprocessed.iloc[:1]["Year"][0])

In [ ]:
df_mag_preprocessed = df_mag_preprocessed.astype({"Year": int}, errors='raise') 
df_mag_preprocessed.iloc[:3]

Fix of the CitationCount data type, that has been interpreted as a float.

In [ ]:
type(df_mag_preprocessed.iloc[:1]["CitationCount"][0])

In [ ]:
df_mag_preprocessed = df_mag_preprocessed.astype({"CitationCount": int}, errors='raise') 
df_mag_preprocessed.iloc[:3]

## Fix of the Missing Conferences Locations
Some papers have only the indication of the conference series. For this reason, the conference instance and the related conference locations don't have a value.

However, every paper has been published in a specific "instance" of a conference, hence it should have a location. These papers will be "fixed" considering the year of their publication and their conference.

In [ ]:
df_mag_preprocessed_subset = df_mag_preprocessed.iloc[:50]
df_mag_preprocessed_subset = df_mag_preprocessed_subset.dropna(subset = ['ConferenceNormalizedName'])
df_mag_preprocessed_subset.iloc[:10][["Year", "ConferenceSeriesNormalizedName", "ConferenceNormalizedName", "ConferenceDisplayName"]]

As you can see in the above test, the ConferenceNormalizedName seems to be made by the concatenation of ConferenceSeriesNormalizedName in lowercase, a space, and the papers' year.

**Note**: in the above subset the ConferenceDisplayName seems to be composed in the same way of ConferenceNormalizedName, but without the lowercase. However, this is not always true!

Now we're going to populate the ConferenceNormalizedName instances that don't have a value.

In [ ]:
df_mag_preprocessed.ConferenceNormalizedName.fillna(df_mag_preprocessed.ConferenceSeriesNormalizedName.str.lower() + ' ' + df_mag_preprocessed.Year.astype(str), inplace=True)
df_mag_preprocessed

I tried to do a new merge with the Conference Instances dataframe (this time it will be made on the ConferenceNormalizedName column), but I had no luck: these conference instances are missing. That's probably the reason of the NaN values in the ConferenceInstanceID field of the original Papers table.

In [ ]:
df_mag_conf_instances.loc[df_mag_conf_instances["ConferenceNormalizedName"] == "enter 2013"]

### Gathering the Missing Conferences Locations from the DBLP Website
The missing conferences locations are going to be obtained from queries to the DBLP Website.

In [ ]:
df_mag_conferences = df_mag_preprocessed[["ConferenceNormalizedName", "ConferenceLocation"]]

In [ ]:
# TODO Test: salvataggio del subset del dataframe per velocizzare le operazioni

# Save on disk
df_mag_conferences.to_csv(path_file_export + 'out_mag_tmp_conferences.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_mag_tmp_conferences.csv')

In [7]:
# TODO Test: lettura del subset del dataframe per velocizzare le operazioni ---> cCODICE DA RIMUOVERE
#df_mag_conferences = pd.read_csv(path_file_export + 'out_mag_tmp_conferences.csv', low_memory=False)
df_mag_conferences = pd.read_csv(path_file_export + 'out_mag_tmp_locations_subset.csv', low_memory=False)

# TODO Test: drop della colonna paper id che non ci serve più ---> cCODICE DA RIMUOVERE
df_mag_conferences = df_mag_conferences[["ConferenceNormalizedName", "ConferenceLocation"]]
df_mag_conferences

,Unnamed: 0,PaperID,ConferenceNormalizedName,ConferenceLocation
0,0,14558443,disc 2014,"Austin, TX"
1,1,15354235,esa 2014,"Wrocław, Poland"
2,2,24327294,enter 2013,NaN
3,3,60437532,dexa 2002,NaN
4,4,198056957,icaisc 2006,NaN
...,...,...,...,...
4409807,4409811,3102242761,iecon 2020,NaN
4409808,4409812,3136855299,bmsb 2020,NaN
4409809,4409813,3145351916,acc 1988,NaN
4409810,4409814,3151696876,icassp 2002,NaN


Drop of the duplicated conferences. We only need unique values.

Drop of the papers that don't need their location to be fixed.

In [11]:
df_mag_conferences = df_mag_conferences[df_mag_conferences["ConferenceLocation"].isna()]
df_mag_conferences

,ConferenceNormalizedName,ConferenceLocation
2,enter 2013,NaN
3,dexa 2002,NaN
4,icaisc 2006,NaN
5,interact 2011,NaN
6,fct 2005,NaN
...,...,...
4397042,dis 2001,NaN
4404554,syscon 2017,NaN
4405987,iwaal 2016,NaN
4407733,pppj 2002,NaN


In [10]:
df_mag_conferences = df_mag_conferences.drop_duplicates(subset="ConferenceNormalizedName")

print(f"Now we only need to search for the location of {df_mag_preprocessed_tmp_locations_subset.__len__()} unique conferences")

Now we only need to search for the location of 40361 unique conferences


Parallel execution of the queries to the DBLP website.

**Note**: this operation should take less than 10min, depending on your Internet speed.

In [ ]:
dblp_url = "https://dblp.org/db/"
dict_conf_locations = {}      
download_list = list(df_mag_conferences.ConferenceNormalizedName.values)

executor = concurrent.futures.ProcessPoolExecutor(max_workers=n_cpu_threads * 10, mp_context=mp_ctx)
futures = [executor.submit(mt_get_mag_conf_location_from_dblp_operation, conf_name, dblp_url) for conf_name in download_list]

for future in concurrent.futures.as_completed(futures):
    try:
        k, v = future.result()
    except Exception as e:
        print(f"{futures[future]} throws {e}")
    else:
        dict_conf_locations[k] = v
        pass

# Converting the resulting dictionary to a dataframe
df_conf_locations = pd.DataFrame(dict_conf_locations.items(), columns=['ConferenceNormalizedName', 'ConferenceLocation'])

Let's see how many conference locations have been fixed.

In [ ]:
tmp_loc = df_conf_locations.copy()
tmp_loc = tmp_loc.dropna(subset = ['ConferenceLocation'])

print(f"Fixed {len(tmp_loc.index)} over {len(df_conf_locations.index)} unique conferences")

tmp_loc = None # Free some memory

Join of the new location data with the original dataframe

In [ ]:
df_mag_preprocessed = pd.merge(df_mag_papers, df_conf_locations, on=['ConferenceNormalizedName'], how='left')
df_mag_preprocessed

Count of how many paper's conference locations are still missing

In [ ]:
n_missing = len(df_mag_preprocessed.dropna(subset = ['ConferenceLocation']).index) - len(df_mag_preprocessed_no_location.index)
print(f"{n_missing} missing locations")

## Write of the Final CSV on Disk

In [ ]:
# Write of the resulting CSV on Disk
df_mag_preprocessed.to_csv(path_file_export + 'out_mag_citations_count_and_conferences.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_mag_citations_count_and_conferences.csv')

Check of the Exported CSV to be sure that everything went fine.

In [ ]:
# Check of the Exported CSV
df_mag_exported_csv = pd.read_csv(path_file_export + 'out_mag_citations_count_and_conferences.csv', low_memory=False)
df_mag_exported_csv

Order by citations count descending to see the articles with the most citations

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_mag_exported_csv = df_mag_exported_csv.sort_values(by='CitationCount', ascending=False)
df_mag_exported_csv